In [0]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("NYC Taxi Fare Prediction").getOrCreate()

In [0]:
from pyspark.sql import functions as F
import requests
import pandas as pd

# Replace with your API Key from Visual Crossing
api_key = '6ZBRRE47XM5UHNVS7X7G53ZU5'
url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/NewYork/2023-01-01/2023-12-31?unitGroup=metric&key={api_key}&include=days"

In [0]:
# List all files in the specified DBFS directory
dbutils.fs.ls("dbfs:/dbfs/FileStore/tables/Data_Processing_Pulic_Holiday/")

[FileInfo(path='dbfs:/dbfs/FileStore/tables/Data_Processing_Pulic_Holiday/_delta_log/', name='_delta_log/', size=0, modificationTime=1732620017548),
 FileInfo(path='dbfs:/dbfs/FileStore/tables/Data_Processing_Pulic_Holiday/part-00000-aac73e49-f7df-476d-a4c9-43498cba8396.c000.snappy.parquet', name='part-00000-aac73e49-f7df-476d-a4c9-43498cba8396.c000.snappy.parquet', size=100300018, modificationTime=1732619883000),
 FileInfo(path='dbfs:/dbfs/FileStore/tables/Data_Processing_Pulic_Holiday/part-00001-a39aa07f-4285-47a9-95dd-16374fa5d23c.c000.snappy.parquet', name='part-00001-a39aa07f-4285-47a9-95dd-16374fa5d23c.c000.snappy.parquet', size=121560585, modificationTime=1732619883000),
 FileInfo(path='dbfs:/dbfs/FileStore/tables/Data_Processing_Pulic_Holiday/part-00002-98f02068-ba39-4099-b7ee-476939fd075f.c000.snappy.parquet', name='part-00002-98f02068-ba39-4099-b7ee-476939fd075f.c000.snappy.parquet', size=127249283, modificationTime=1732619883000),
 FileInfo(path='dbfs:/dbfs/FileStore/tables/

In [0]:
# Load the Delta table
taxi_with_holidays_df = spark.read.format("delta").load("dbfs:/dbfs/FileStore/tables/Data_Processing_Pulic_Holiday/")

# Show the first few rows to verify
display(taxi_with_holidays_df.limit(5))

DOLocationID,PULocationID,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,payment_type,fare_amount,tolls_amount,improvement_surcharge,total_amount,trip_duration,pickup_day_of_week,pickup_hour,Month_Num,pickup_month,dropoff_hour,dropoff_day_of_week,dropoff_month,dropoff_week_of_year,pickup_date,pickup_time,dropoff_date,dropoff_time,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,distance_km,pickup_borough,pickup_zone,pickup_service_zone,LocationID,dropoff_borough,dropoff_zone,dropoff_service_zone,holidayName,normalizeHolidayName,is_holiday
50,43,2023-10-09T09:35:28Z,2023-10-09T09:45:20Z,3,1.7,2,10.7,0.0,1.0,14.7,9.866667,2,9,10,10,9,2,10,41,2023-10-09,09:35:28,2023-10-09,09:45:20,40.782478099747884,-73.9655535654591,40.76623772504185,-73.99513522075515,3.0766876214201546,Manhattan,Central Park,Yellow Zone,50,Manhattan,Clinton West,Yellow Zone,Columbus Day,Columbus Day,1
236,43,2023-10-09T11:08:31Z,2023-10-09T11:31:08Z,1,1.63,1,19.8,0.0,1.0,30.94,22.616667,2,11,10,10,11,2,10,41,2023-10-09,11:08:31,2023-10-09,11:31:08,40.782478099747884,-73.9655535654591,40.780436437189955,-73.95701169835736,0.7541840936610109,Manhattan,Central Park,Yellow Zone,236,Manhattan,Upper East Side North,Yellow Zone,Columbus Day,Columbus Day,1
48,43,2023-10-09T12:11:13Z,2023-10-09T12:34:21Z,1,2.8,1,18.4,0.0,1.0,26.85,23.133333,2,12,10,10,12,2,10,41,2023-10-09,12:11:13,2023-10-09,12:34:21,40.782478099747884,-73.9655535654591,40.76225275531935,-73.98984464313298,3.040063819125508,Manhattan,Central Park,Yellow Zone,48,Manhattan,Clinton East,Yellow Zone,Columbus Day,Columbus Day,1
142,43,2023-10-09T13:26:51Z,2023-10-09T13:39:11Z,1,1.87,1,13.5,0.0,1.0,21.88,12.333333,2,13,10,10,13,2,10,41,2023-10-09,13:26:51,2023-10-09,13:39:11,40.782478099747884,-73.9655535654591,40.77363329302708,-73.98153220639182,1.6665688371988927,Manhattan,Central Park,Yellow Zone,142,Manhattan,Lincoln Square East,Yellow Zone,Columbus Day,Columbus Day,1
43,43,2023-10-09T16:57:14Z,2023-10-09T16:58:32Z,1,0.2,2,3.7,0.0,1.0,7.7,1.3,2,16,10,10,16,2,10,41,2023-10-09,16:57:14,2023-10-09,16:58:32,40.782478099747884,-73.9655535654591,40.782478099747884,-73.9655535654591,0.0,Manhattan,Central Park,Yellow Zone,43,Manhattan,Central Park,Yellow Zone,Columbus Day,Columbus Day,1


In [0]:
# Fetch weather data
response = requests.get(url)

if response.status_code == 200:
    weather_data = response.json()
    # Convert the JSON response to Pandas DataFrame
    weather_df = pd.json_normalize(weather_data['days'])
    weather_df = pd.DataFrame(weather_df[['datetime', 'temp']])  # Select only date and temperature

    # Convert to Spark DataFrame
    weather_spark_df = spark.createDataFrame(weather_df)
    # Rename and format the date column
    weather_spark_df = weather_spark_df.withColumnRenamed('datetime', 'weather_date')
    weather_spark_df = weather_spark_df.withColumn("weather_date", F.to_date("weather_date"))

    # Join the weather data with taxi data on date columns
    taxi_with_weather_df = taxi_with_holidays_df.join(
        weather_spark_df,
        taxi_with_holidays_df["pickup_date"] == weather_spark_df["weather_date"],
        how="left"
    )

    # Drop the duplicate date column from weather data if needed
    taxi_with_weather_df = taxi_with_weather_df.drop("weather_date")

    # Show the updated DataFrame with weather info
    taxi_with_weather_df.show()
else:
    print("Failed to fetch weather data. Status code:", response.status_code)

In [0]:
display(taxi_with_weather_df)

In [0]:
# Function to display value counts for a specific column using display()
def display_value_counts(dataframe, column):
    print(f"Value counts for {column}:")
    value_counts = dataframe.groupBy(column).count().orderBy("count", ascending=False)
    display(value_counts)

# Example usage for a specific column
display_value_counts(taxi_with_weather_df, 'temp')

Value counts for temp:


temp,count
24.4,1289333
25.0,1009064
24.8,880655
25.6,762028
26.2,730434
23.8,678086
25.1,669150
26.3,667854
24.5,607596
26.5,589325


In [0]:
num_rows = taxi_with_weather_df.count()
num_cols = len(taxi_with_weather_df.columns)
print(f"Shape: ({num_rows}, {num_cols})")


Shape: (36395607, 40)


In [0]:
# Check for Missing Values in each column
# Check for missing values
missing_values = taxi_with_weather_df.select([((taxi_with_weather_df[column].isNull()).cast("int")).alias(column) for column in taxi_with_weather_df.columns]) \
                                .agg({column: 'sum' for column in taxi_with_weather_df.columns})

display(missing_values.limit(5))

sum(dropoff_time),sum(DOLocationID),sum(improvement_surcharge),sum(pickup_hour),sum(PULocationID),sum(trip_distance),sum(dropoff_borough),sum(pickup_service_zone),sum(pickup_date),sum(dropoff_longitude),sum(LocationID),sum(pickup_latitude),sum(dropoff_day_of_week),sum(tolls_amount),sum(dropoff_date),sum(pickup_time),sum(Month_Num),sum(pickup_zone),sum(pickup_borough),sum(payment_type),sum(fare_amount),sum(pickup_longitude),sum(temp),sum(passenger_count),sum(dropoff_month),sum(dropoff_hour),sum(pickup_day_of_week),sum(dropoff_zone),sum(distance_km),sum(dropoff_datetime),sum(normalizeHolidayName),sum(dropoff_week_of_year),sum(trip_duration),sum(total_amount),sum(holidayName),sum(pickup_datetime),sum(dropoff_latitude),sum(is_holiday),sum(dropoff_service_zone),sum(pickup_month)
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# Check for Duplicates
# Count the number of duplicate rows
duplicates_count = taxi_with_weather_df.count() - taxi_with_weather_df.distinct().count()
print(f"Number of duplicate rows: {duplicates_count}")

Number of duplicate rows: 0


In [0]:
taxi_with_weather_df.show(5)

+------------+------------+-------------------+-------------------+---------------+-------------+------------+-----------+------------+---------------------+------------+-------------+------------------+-----------+---------+------------+------------+-------------------+-------------+--------------------+-----------+-----------+------------+------------+------------------+-----------------+------------------+------------------+------------------+--------------+------------+-------------------+----------+---------------+--------------------+--------------------+------------+--------------------+----------+----+
|DOLocationID|PULocationID|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|payment_type|fare_amount|tolls_amount|improvement_surcharge|total_amount|trip_duration|pickup_day_of_week|pickup_hour|Month_Num|pickup_month|dropoff_hour|dropoff_day_of_week|dropoff_month|dropoff_week_of_year|pickup_date|pickup_time|dropoff_date|dropoff_time|   pickup_latitude| pickup

In [0]:
# Define a more descriptive Delta Lake storage path
delta_path = "/dbfs/FileStore/tables/cleaned_nyc_taxi_fare/"
# Write the DataFrame to Delta format
taxi_with_weather_df.write.format("delta").mode("overwrite").save(delta_path)